This snippet illustrates that we should use variable_scope when we are dealing with variables, and more important when we plan to share variables.

In a general case, name_scope will do.


** My takeaway ** - whenever you create variables, create variables using tf.get_variable and scope it within tf.variable_scope. 

In [2]:
import tensorflow as tf

In [3]:
with tf.name_scope("scope1"):
    with tf.variable_scope("var"):
        x_a = tf.get_variable("xa", shape=[1,2], initializer=tf.constant_initializer(1))
        x_b = tf.get_variable("xb", shape=[1,2], initializer=tf.constant_initializer(2))

In [4]:
# note this will cause error: even if they belong to different name_scopes - scope1 and scope2, 
# get_variable only respects variable_scope - so it will search for var/xa and va/xb which are already defined
'''
with tf.name_scope("scope2"):
    with tf.variable_scope("var"):
        y_a = tf.get_variable("xa", shape=[1,2], initializer=tf.constant_initializer(10))
        y_b = tf.get_variable("xb", shape=[1,2], initializer=tf.constant_initializer(20))
'''

# the following will work because variable_scope will search for scope2/var/xa and scope2/var/xb
with tf.variable_scope("scope2"):
    with tf.variable_scope("var"):
        y_a = tf.get_variable("xa", shape=[1,2], initializer=tf.constant_initializer(10))
        y_b = tf.get_variable("xb", shape=[1,2], initializer=tf.constant_initializer(20))



In [5]:
with tf.name_scope("scope3"):
    with tf.variable_scope("var", reuse=True):
        z_a = tf.get_variable("xa", shape=[1,2])
        z_b = tf.get_variable("xb", shape=[1,2])

### Small side-track experiment

In [7]:
# the following will fail because var/xa is defined
'''
with tf.name_scope("scope4"):
    with tf.name_scope("scope4deeper"):
        with tf.variable_scope("var"):
            zz_a = tf.get_variable("xa", shape=[1,2])
'''            

'\nwith tf.name_scope("scope4"):\n    with tf.name_scope("scope4deeper"):\n        with tf.variable_scope("var"):\n            zz_a = tf.get_variable("xa", shape=[1,2])\n'

In [8]:
with tf.name_scope("scope5"):
    with tf.variable_scope("dummy"):
        with tf.name_scope("scope5deeper"):
            with tf.variable_scope("var"):
                zzz_a = tf.get_variable("xa", shape=[1,2])

In [10]:
# note how only variable_scope matters for get_variable
print(zzz_a)

<tf.Variable 'dummy/var/xa:0' shape=(1, 2) dtype=float32_ref>


### Back to variable sharing

In [5]:
sum_1 = x_a + x_b
sum_2 = y_a + y_b
sum_3 = z_a + z_b

In [7]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter("logs", sess.graph)
    sess.run(tf.global_variables_initializer())
    print(sess.run([sum_1, sum_2, sum_3]))
    print("$$$$$$$$$$$###############$$$$$$$$$$$$$4")
    print(sess.graph_def)

[array([[ 3.,  3.]], dtype=float32), array([[ 30.,  30.]], dtype=float32), array([[ 3.,  3.]], dtype=float32)]
$$$$$$$$$$$###############$$$$$$$$$$$$$4
node {
  name: "var/xa/Initializer/Const"
  op: "Const"
  attr {
    key: "_class"
    value {
      list {
        s: "loc:@var/xa"
      }
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 1
          }
          dim {
            size: 2
          }
        }
        float_val: 1.0
      }
    }
  }
}
node {
  name: "var/xa"
  op: "VariableV2"
  attr {
    key: "_class"
    value {
      list {
        s: "loc:@var/xa"
      }
    }
  }
  attr {
    key: "container"
    value {
      s: ""
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 1
        }
        dim {


**Note below how variable_scoping stands out**

<img src="files/variable_sharing.png">